In [ ]:
__author__ = "Nantia Leonidou"
__description__ = " Run pymCADRE "

from rank.rank_reactions import *
from prune.prune_model import *


    Inputs:
        - model: original generic model
        - precursorMets: name of .txt file with precursor metabolites as string
        - G: list of genes in expression data
        - U: ubiquity scores corresponding to genes in G
        - confidenceScores: literature-based evidence for generic model reactions
        - salvageCheck: option flag for whether to perform functional check for the
                        nucleotide salvage pathway (1) or not (0)
        - C_H_genes: predefined high confidence reactions (optional)
        - method: 1 = use fastFVA (glpk) to check consistency; 2 = use fastcc & cplex

    Outputs:
        - PM: pruned, context-specific model
        - GM: generic model (after removing blocked reactions)
        - C: core reactions in GM
        - NC: non-core reactions in GM
        - Z: reactions with zero expression (i.e., measured zero, not just
             missing from expression data)
        - model_C: core reactions in the original model (including blocked)
        - pruneTime: total reaction pruning time
        - cRes: result of model checks (consistency/function)
            - vs. +: reaction r removed from generic model or not
            1 vs. 2: reaction r had zero or non-zero expression evidence
            -x.y: removal of reaction r corresponded with removal of y (num.) total core reactions
            +x.1 vs. x.0: precursor production possible after removal of reaction r or not
            3: removal of reaction r by itself prevented production of required metabolites (therefore was not removed)  


## Import Sample Dataset

In [ ]:
import pandas as pd
import numpy as np
# model
model = io.read_sbml_model('dataset/RECON1.xml')
# genes
G = pd.read_csv('../pre_processing/dataset/1_GPL570_GSE3397/1_GPL570_GSE3397_entrez_ids.csv')
G = list(G['ENTREZ_GENE_ID'])
# ubiquity scores
U = pd.read_csv('../pre_processing/dataset/1_GPL570_GSE3397/1_GPL570_GSE3397_ubiquity.csv', header=None)
U = U.rename(columns={0: "Scores"})
U = list(U['Scores'])
# confidence scores
confidence_scores = pd.read_csv('../pre_processing/dataset/Recon1_confidence_scores.csv')
confidence_scores = np.float64(list(confidence_scores['Confidence Score']))
# list with precursor metabolites
precursorMets = '../pre_processing/dataset/key_metabolites_RECON1.txt'

## Ranking

In [ ]:
##############################################
# Generate order for reaction removal
##############################################
# Gene ubiquity scores are converted to reaction expression evidence to
# define the core (C) and non-core (NC) reaction sets. Inactive reactions
# are identified and removed from the global model to produce the generic
# model (GM) for subsequent pruning. Non-core reactions are ordered first
# by expression and then by connectivity evidence to give the list P. Any
# reactions with zero expression (i.e., associated, but non-expressed
# genes) are also listed in the vector Z.

print('Processing inputs and ranking reactions...')
GM, C, NC, P, Z, model_C = rank_reactions(model, G, U, confidence_scores, [], method=1)

In [ ]:
print(len(GM.reactions),len(GM.metabolites),len(GM.genes),len(C),len(NC),len(Z),len(model_C))

## Check model consistency

In [ ]:
##################################################
# Define inputs to the model pruning step
##################################################
# Define core vs. non-core ratio threshold for removing reactions
eta = 1 / 3
# Check functionality of generic model
genericStatus = check_model_function(GM, 'required_mets', precursorMets)[0]


## Pruning

In [ ]:
if genericStatus:
    print('Generic model passed precursor metabolites test')

    ##############################################################
    # If generic functionality test is passed, prune reactions
    ###############################################################
    print('Pruning reactions...')
    t0 = process_time()
    PM, cRes = prune_model(GM, P, C, Z, eta, precursorMets, salvage_check=1, method=1)
    # Stop the stopwatch / counter
    t_stop = process_time()
    # compute elapsed time
    pruneTime = t_stop - t0

else:
    print('Generic model failed precursor metabolites test!!')

In [ ]:
print(len(PM.reactions),len(PM.metabolites),len(PM.genes),pruneTime)

## Store pruned model

In [ ]:
#########################################
# * store pruned model in SBML format *
#########################################
io.write_sbml_model(PM, "pruned_model.xml")
